In [1]:
%matplotlib inline

from fastai.vision import *
import stable.predict
from tqdm import tqdm_notebook as tqdm

In [2]:
path = Path("data")
train_fnames = (path/"train_fnames.txt").read_text().split("\n")
valid_fnames = (path/"valid_fnames.txt").read_text().split("\n")
df_train = pd.read_csv(path/"train.csv")
unicode_map = {codepoint: char for codepoint, char in pd.read_csv('data/unicode_translation.csv').values}

In [23]:
bbox_prediction_path = ""
unlabelled = pd.read_csv(bbox_prediction_path)

In [24]:
unlabelled

,image_id,labels
0,test_00145af3,nan 588 608 78 81 nan 1408 612 91 70 nan 104 6...
1,test_001c37e2,nan 1024 756 57 62 nan 1176 772 19 77 nan 1316...
2,test_003aa33a,nan 840 748 23 51 nan 1156 756 36 65 nan 520 7...
3,test_00665e33,nan 488 548 57 81 nan 1216 548 103 86 nan 636 ...
4,test_006964dc,nan 1784 580 96 14 nan 464 596 35 89 nan 636 5...
...,...,...
4145,test_ffb1f141,nan 1376 716 17 58 nan 1872 732 24 66 nan 1196...
4146,test_ffe0bb66,nan 520 536 76 67 nan 340 540 103 102 nan 984 ...
4147,test_fff039a9,nan 1820 792 42 91 nan 2264 804 48 139 nan 138...
4148,test_fff50dbc,nan 1320 592 37 76 nan 1796 592 71 40 nan 1648...


In [29]:
classifier = load_learner(path=path/"train_images_chars_256",
                          file="20190925_TEST").to_fp32()

In [31]:
for i, row in tqdm(unlabelled.iterrows()):
    fname = row.image_id
    im = PIL.Image.open(path/"test_images"/f"{fname}.jpg")
    im_arr = np.array(im)
    im_height, im_width, colour_depth = im_arr.shape 
    
    R = 128
    
    predictions = []    
    if type(row.labels) == str:
        lbls = np.array(row.labels.split()).reshape(-1,5)
        for _, center_x, center_y, width, height in lbls:
            center_x, center_y = int(center_x), int(center_y)
            width, height = int(width), int(height)
            if width < 5 or height < 5:
                # Ignore from prediction
                print(fname, center_x, center_y, width, height)
            else:
                x0, y0 = max(0, center_x - R), max(0, center_y - R)
                x1, y1 = center_x + R, center_y + R
                character_region = im_arr[y0:y1, x0:x1]
                prediction = classifier.predict(stable.predict.preprocess_input(character_region))[0].obj
                if prediction != "BACKGROUND":
                    predictions.append([prediction, center_x, center_y])
            
    if len(predictions) > 0:
        predictions_str = " ".join([" ".join(map(str, i)) for i in predictions])
        unlabelled.iloc[i].labels = predictions_str
    else:
        unlabelled.iloc[i].labels = np.nan

test_01e41977 820 2924 3 6
test_021661ed 1416 1208 108 0
test_021661ed 308 1912 139 4
test_051829d5 248 1392 180 0
test_051829d5 732 4796 42 0
test_051829d5 376 4812 218 0
test_071bf9b6 292 1604 188 1
test_0778b88d 1192 2876 78 0
test_08e0ba73 2224 3768 211 0
test_09f22cd6 1068 4532 175 0
test_0a9bab86 840 1348 160 0
test_0a9bab86 396 2580 115 0
test_0a9bab86 1352 3660 80 0
test_0adbe8a5 2268 2508 184 4
test_0adbe8a5 352 4724 168 0
test_0dc6ea9a 904 1628 117 2
test_145a0fc4 1832 776 3 8
test_14d8e349 2272 2508 89 0
test_1ada762d 1728 1336 0 0
test_1ada762d 2528 2896 167 1
test_1b2235cf 2228 2812 133 0
test_1b612d1f 1184 4284 178 1
test_1d9aebbc 2920 1516 150 0
test_1e4be7b4 1108 1756 164 0
test_1e4be7b4 2656 3504 106 0
test_1e552627 692 1940 43 1
test_1ed95720 420 2308 153 0
test_1ed95720 2680 2492 175 0
test_213b4f83 1684 1820 184 0
test_213b4f83 2960 2284 149 0
test_21cdd6dc 1788 1364 45 4
test_228bff59 772 1440 170 0
test_228bff59 2668 4844 144 0
test_233bc8ae 448 1896 3 13
test_240

In [32]:
submission_no = 20

In [33]:
unlabelled.to_csv(f"submissions/submission{submission_no}.csv", index=False)